In [1]:
from random import choice
from pathlib import Path

In [3]:
from spleeter.audio.adapter import get_default_audio_adapter
from spleeter.separator import Separator
from spleeter.audio.convertor import to_stereo

In [4]:
import audioread
from librosa.core import load

In [5]:
import IPython.display as ipd

In [6]:
current_dir = Path('.')

Данные находятся на 2 уровня выше (если не так, то нужно прописать путь по папки)

In [11]:
data_dir = current_dir / '..' / '..' / 'data' / 'nr' / "train"

In [12]:
list(data_dir.iterdir())

[WindowsPath('../../data/nr/train/mix'),
 WindowsPath('../../data/nr/train/source')]

В `mix` находятся смешанные дорожки, а в `source` - исходники.  
Выберем произвольную дорожку и посмотрим на формат данных

In [36]:
random_folder = choice(list((data_dir / 'mix').iterdir())).stem

In [37]:
random_folder

'7e1a79eed4024ce69b66bfbf61811f33'

In [38]:
mix_audio_path= data_dir / 'mix' / random_folder / "mix.wav"  
speech_audio_path = data_dir / 'source' / random_folder / "1.wav"
noise_audio_path = data_dir / 'source' / random_folder / "2.wav"

In [39]:
with audioread.audio_open(str(mix_audio_path)) as f:
    print(f.channels, f.samplerate, f.duration)

1 8000 15.0


Моно, относительно низкий sample rate

In [40]:
sample_rate = 8000

In [41]:
mix_audio, _ = load(str(mix_audio_path), sample_rate)
speech_audio, _ = load(str(speech_audio_path), sample_rate)
noise_audio, _ = load(str(noise_audio_path), sample_rate)

Давайте посмотрим, как работает обычная модель с этими данными

In [42]:
adapter = get_default_audio_adapter()

In [43]:
waveform, _ = adapter.load(str(mix_audio_path), sample_rate=sample_rate)

In [44]:
if not waveform.shape[-1] == 2:
    waveform = to_stereo(waveform)

In [45]:
separator = Separator('spleeter:2stems')

In [46]:
predictor = separator._get_predictor()

INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models\\2stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001B335A67608>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Apply unet for vocals_spectrogram
INFO:tensorflow:Apply unet for accompaniment_spectrogram
INFO:tensorflow:Done calling 

In [47]:
prediction = predictor({
            'waveform': waveform,
            'audio_id': ''})
_ = prediction.pop('audio_id')

In [48]:
predicted_voice_path = f'output/{random_folder}_vocals.wav'
adapter.save(predicted_voice_path, prediction['vocals'], sample_rate)

INFO:spleeter:File output/7e1a79eed4024ce69b66bfbf61811f33_vocals.wav written


In [49]:
predicted_noise_path = f'output/{random_folder}_accompaniment.wav'
adapter.save(predicted_noise_path, prediction['accompaniment'], sample_rate)

INFO:spleeter:File output/7e1a79eed4024ce69b66bfbf61811f33_accompaniment.wav written


Послушаем результаты

#### 1. Оригинальная шумная дорожка

In [54]:
ipd.Audio(mix_audio, rate=sample_rate)

#### 2.1 Голос, очищенный моделью

In [55]:
ipd.Audio(predicted_voice_path, rate=sample_rate)

#### 2.2 Оригинальный голос

In [56]:
ipd.Audio(speech_audio, rate=sample_rate)

#### 3.1 Шум, который отделила модель

In [57]:
ipd.Audio(predicted_noise_path, rate=sample_rate)

#### 3.2 Оригинальный шум

In [59]:
ipd.Audio(noise_audio, rate=sample_rate)

± ок, только чрезмерно приглушенно